# Import

In [1]:
from functools import reduce
from pathlib import Path
import time
import datetime

import pandas as pd
import requests
from wmfdata import hive
from wmfdata.utils import print_err, pd_display_all

# Parameters

In [2]:
# TSV file where metrics are or will be saved
FILENAME = "metrics/metrics.tsv"

# Metric month. The mediawiki_history snapshot must be from the metrics month or later.
last_month = datetime.date.today().replace(day=1) - datetime.timedelta(days=1)

METRICS_MONTH_TEXT = last_month.strftime("%Y-%m")
MEDIAWIKI_HISTORY_SNAPSHOT = last_month.strftime("%Y-%m")



In [3]:
datetime.date.today().replace(day=1) - datetime.timedelta(days=1)

datetime.date(2020, 3, 31)

# Preparation

In [4]:
# Convert our metrics month to all the formats we need and provide them in a dict
# so we can easily use them to format strings
metrics_month = METRICS_MONTH_TEXT
date_params = {
    "mediawiki_history_snapshot": MEDIAWIKI_HISTORY_SNAPSHOT,
    "metrics_month": str(metrics_month),
    #"metrics_prev_month": str(metrics_month - 1),
    #"metrics_month_start": str(metrics_month.start_time), 
    "metrics_month_first_day": str((datetime.date.today()- datetime.timedelta(days=31)).replace(day=1)),
    #"metrics_month_end": str(last_month),
    "metrics_month_last_day": str(last_month),
    "metrics_year": last_month.year,
    "metrics_cur_month" : last_month.month
}

# Load any previous results
try:
    old_metrics = (
        pd.read_csv(FILENAME, sep="\t", parse_dates = ["month"])
        .set_index("month")
    )
except FileNotFoundError:
    old_metrics = None
    
def prepare_query(filename):
    return (
        Path(filename)
        .read_text()
        .format(**date_params)
    )

# MariaDB and Hive query metrics


In [5]:
queries = {
    "pageviews": {
        "file": "queries/pageviews.hql",
        "engine": "hive"
    },
    "page_previews": {
        "file": "queries/page_previews.hql",
        "engine": "hive"
    },
    "unique_devices": {
        "file": "queries/unique_devices.hql",
        "engine": "hive"
    },
    "global_south_pageviews": {
        "file": "queries/global_south_pageviews.hql",
        "engine": "hive"
    },
    "mobile-heavy_pageviews": {
        "file": "queries/mobile_heavy_pageviews.hql",
        "engine": "hive"
    },
    "mobile-heavy_previews": {
        "file": "queries/mobile_heavy_previews.hql",
        "engine": "hive"
    },
    "global_south_previews": {
        "file": "queries/global_south_previews.hql",
        "engine": "hive"
    }
       
}


for key, val in queries.items():
    query = prepare_query(val["file"])
    engine = val["engine"]
    print_err("Running {} on {}...".format(key, engine))
    
    if engine == "mariadb":
        result = mariadb.run(query)
    elif engine == "hive":
        result = hive.run(query)
    else:
        raise ValueError("Unknown engine specified.") 
    
    result = result.assign(month=lambda df: pd.to_datetime(df["month"]))
    val["result"] = result

Running global_north_previews on hive...
Running pageviews on hive...
Running page_previews on hive...
Running mobile-heavy_previews on hive...
Running unique_devices on hive...
Running global_south_previews on hive...
Running mobile-heavy_pageviews on hive...
Running global_south_pageviews on hive...
Running global_north_pageviews on hive...


# Combining and saving metrics

In [6]:
# Assemble list of result dataframes
results = [val["result"] for _, val in queries.items()]

# Merge them all, assuming that the month is the only common column
new_metrics = reduce(lambda l, r: pd.merge(l, r, how="outer"), results)

# Set the month as an index so combine_first works properly
new_metrics = new_metrics.set_index("month").sort_index()

# Add Metrics for Interactions
new_metrics['interactions'] = new_metrics.apply(lambda x: x['previews_seen'] + x['total_pageview'], axis=1)
new_metrics['gs_interactions'] = new_metrics.apply(lambda x: x['gs_previews'] + x['gs_pageviews'], axis=1)
new_metrics['mh_interactions'] = new_metrics.apply(lambda x: x['mh_previews'] + x['mh_pageviews'], axis=1)


In [8]:
if old_metrics is None:
    metrics = new_metrics
else:
    metrics = new_metrics.combine_first(old_metrics)
    
pd_display_all(metrics.tail(10))

,desktop,gn_interactions,gn_pageviews,gn_previews,gs_interactions,gs_pageviews,gs_previews,interactions,mh_interactions,mh_pageviews,mh_previews,mobileweb,previews_seen,total_pageview,unique_devices
month,,,,,,,,,,,,,,,
2019-06-01,5.790501e+09,1.261947e+10,1.135743e+10,1.262031e+09,3.785372e+09,3.442175e+09,343196693.0,1.644978e+10,617333603.0,588140293.0,29193310.0,8.804954e+09,1.609283e+09,1.484050e+10,1.506536e+09
2019-07-01,5.894085e+09,1.303928e+10,1.176214e+10,1.277139e+09,3.913795e+09,3.552436e+09,361359372.0,1.702111e+10,687542306.0,655773508.0,31768798.0,9.226142e+09,1.643555e+09,1.537756e+10,1.502241e+09
2019-08-01,5.718161e+09,1.300368e+10,1.173426e+10,1.269425e+09,3.986866e+09,3.624499e+09,362366891.0,1.706324e+10,723552995.0,692032091.0,31520904.0,9.445959e+09,1.636496e+09,1.542674e+10,1.509795e+09
2019-09-01,5.889674e+09,1.312222e+10,1.174346e+10,1.378752e+09,4.131243e+09,3.741501e+09,389742454.0,1.731813e+10,769139828.0,735344494.0,33795334.0,9.408033e+09,1.773316e+09,1.554482e+10,1.695557e+09
2019-10-01,6.714077e+09,1.428526e+10,1.275837e+10,1.526889e+09,4.311099e+09,3.906008e+09,405090073.0,1.865877e+10,825074721.0,788811634.0,36263087.0,9.750824e+09,1.936367e+09,1.672240e+10,1.811302e+09
2019-11-01,6.404464e+09,1.399369e+10,1.247911e+10,1.514581e+09,4.096324e+09,3.705914e+09,390410201.0,1.814010e+10,787954594.0,755302852.0,32651742.0,9.572732e+09,1.909127e+09,1.623097e+10,1.694747e+09
2019-12-01,5.988201e+09,1.368632e+10,1.224315e+10,1.443172e+09,3.724967e+09,3.364769e+09,360197742.0,1.745631e+10,758927001.0,723749087.0,35177914.0,9.388243e+09,1.807163e+09,1.564915e+10,1.576535e+09
2020-01-01,6.598842e+09,1.515085e+10,1.345590e+10,1.694953e+09,3.998114e+09,3.601104e+09,397009855.0,1.919971e+10,819825311.0,782726990.0,37098321.0,1.020693e+10,2.097220e+09,1.710249e+10,1.639562e+09
2020-02-01,6.144843e+09,1.381909e+10,1.226581e+10,1.553277e+09,3.805531e+09,3.431799e+09,373731960.0,1.767025e+10,765111783.0,731187027.0,33924756.0,9.327580e+09,1.931595e+09,1.573866e+10,1.576108e+09


In [9]:
metrics.to_csv(FILENAME, sep="\t")